# Yelp Checkins Wrangling

In [1]:
%matplotlib inline

import pandas as pd
import datetime
import json
import numpy as np

DRY_RUN = False

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
day_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def time_marker(text=''):
    print('[{}] {}'.format(datetime.datetime.now().time(), text.lower()))
    
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

# Load Checkins Data

In [4]:
time_marker(text='Loading Check Ins Data...')

data = pd.DataFrame()
source_data_file = '../source_data/checkin.json'

checkins_list = []
for line in open(source_data_file, 'r'):
    checkins_list.append(json.loads(line))

time_marker(text='creating dataframe...')
checkins_df = pd.DataFrame(checkins_list)
checkins_df.head(3)

[23:00:50.137804] loading check ins data...
[23:00:54.354004] creating dataframe...


,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Thursday': {'21:00': 4, '1:00': 1, '4:00': 1..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Monday': {'13:00': 1}, 'Thursday': {'20:00':..."
2,tJRDll5yqpZwehenzE2cSg,"{'Monday': {'12:00': 1, '1:00': 1}, 'Saturday'..."


In [5]:
business_ids = checkins_df.business_id.to_frame()
business_ids.head(3)

,business_id
0,7KPBkxAOEtb3QeIL9PEErg
1,kREVIrSBbtqBhIYkTccQUg
2,tJRDll5yqpZwehenzE2cSg


# Unpack `time` column

In [6]:
time_marker(text='unpacking daily check in counts...')
checkins_df = unpack(checkins_df, 'time')
checkins_df.head(3)

[23:00:54.720194] unpacking daily check in counts...


,business_id,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,7KPBkxAOEtb3QeIL9PEErg,"{'16:00': 1, '14:00': 2, '10:00': 2, '23:00': ...","{'12:00': 1, '11:00': 1, '14:00': 1, '18:00': ...","{'21:00': 1, '23:00': 3, '18:00': 4, '10:00': ...","{'18:00': 1, '16:00': 1, '14:00': 1, '19:00': ...","{'21:00': 4, '1:00': 1, '4:00': 1, '2:00': 1, ...","{'18:00': 2, '12:00': 1, '13:00': 2, '16:00': ...","{'11:00': 2, '13:00': 2, '14:00': 1, '17:00': ..."
1,kREVIrSBbtqBhIYkTccQUg,NaN,{'13:00': 1},"{'21:00': 1, '16:00': 1}",{'19:00': 1},"{'20:00': 1, '13:00': 1}",NaN,{'17:00': 1}
2,tJRDll5yqpZwehenzE2cSg,NaN,"{'12:00': 1, '1:00': 1}",{'16:00': 1},NaN,{'0:00': 1},NaN,NaN


### If this is a dry run, only process the first 20 business records

In [7]:
if DRY_RUN:
    checkins_df = checkins_df[:20].copy()

# Unpack Hourly Checkins from each Day Column

In [8]:
time_marker(text='splitting daily counts into hourly columns...')
chunks = []

for day in checkins_df.columns[1:]:
    time_marker('unstacking {}...'.format(day))
    data = checkins_df[day].copy().to_frame()

    # fill missing daily checkin dicts with empty dict
    data=data.applymap(lambda x: {} if pd.isnull(x) else x)

    # unpack dict to hourly
    chunk = data[day].apply(pd.Series)

    chunk.columns = [int(str(x).split(':')[0]) for x in chunk.columns]

    # fill in missing hours
    for n in range(0, 24, 1):
        if n not in chunk.columns:
            chunk[n] = np.nan
    chunk['day'] = day

    chunks.append(chunk)

checkins_df = pd.concat(chunks)
checkins_df.fillna(0, inplace=True)
checkins_df.head(3)

[23:00:55.356745] splitting daily counts into hourly columns...
[23:00:55.357448] unstacking friday...
[23:01:58.286707] unstacking monday...
[23:03:00.242532] unstacking saturday...
[23:04:01.903390] unstacking sunday...
[23:05:03.474791] unstacking thursday...
[23:06:03.999049] unstacking tuesday...
[23:07:04.231345] unstacking wednesday...


,0,10,11,12,13,14,15,16,17,18,...,23,2,3,4,5,6,7,8,9,day
0,2.0,2.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Friday
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Friday
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Friday


# Append `dayofweek` to each column

In [9]:
time_marker('appending numerical dayofweek to each row...')
checkins_df['dayofweek'] = checkins_df.apply(lambda row: day_labels.index(row.day.title()), axis=1)
checkins_df.drop('day', axis=1, inplace=True)

[23:08:05.658040] appending numerical dayofweek to each row...


# Append `business_id` to each daily checkin row, left merge

In [10]:
time_marker(text='appending business_id columns to hourly records...')
checkins_clean = checkins_df.merge(business_ids, left_index=True, right_index=True, how='left')
checkins_clean.head(8)

[23:08:37.816239] appending business_id columns to hourly records...


,0,10,11,12,13,14,15,16,17,18,...,2,3,4,5,6,7,8,9,dayofweek,business_id
0,2.0,2.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,7KPBkxAOEtb3QeIL9PEErg
0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7KPBkxAOEtb3QeIL9PEErg
0,1.0,1.0,0.0,1.0,3.0,1.0,1.0,2.0,3.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,7KPBkxAOEtb3QeIL9PEErg
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,6,7KPBkxAOEtb3QeIL9PEErg
0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3,7KPBkxAOEtb3QeIL9PEErg
0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,7KPBkxAOEtb3QeIL9PEErg
0,2.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2,7KPBkxAOEtb3QeIL9PEErg
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,kREVIrSBbtqBhIYkTccQUg


# Cleanup and Reset Index

In [11]:
time_marker(text='cleaning up and reset index...')
checkins_clean.fillna(0, inplace=True)

checkins_clean.sort_values(['business_id', 'dayofweek'], inplace=True)

checkins_clean.reset_index(inplace=True, drop=True)
checkins_clean.head(8)

[23:08:38.497716] cleaning up and reset index...


,0,10,11,12,13,14,15,16,17,18,...,2,3,4,5,6,7,8,9,dayofweek,business_id
0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,--6MefnULPED_I942VcFNA
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,--6MefnULPED_I942VcFNA
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,--6MefnULPED_I942VcFNA
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3,--6MefnULPED_I942VcFNA
4,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,--6MefnULPED_I942VcFNA
5,3.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,--6MefnULPED_I942VcFNA
6,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,6,--6MefnULPED_I942VcFNA
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,--7zmmkVg-IMGaXbuVd0SQ


# Group by `business_id`

In [12]:
time_marker('group by business_id....')
checkins_clean = checkins_clean.groupby(['business_id', 'dayofweek']).sum().unstack()

# relabel columns day_hour
time_marker('relabeling columns...')
corrected_columns = ['d{:d}_h{:d}'.format(day, hour)for hour in range(0, 24) for day in range(0, 7)]
checkins_clean.columns = corrected_columns

# sort columns
time_marker('sorting columns...')
sorted_columns = ['d{:d}_h{:d}'.format(day, hour) for day in range(0, 7) for hour in range(0, 24)]

# apply sort
checkins_clean = checkins_clean[sorted_columns]

# add sum column for sorting
time_marker('collecting total count of checkins for business...')
checkins_clean['total_checkins'] = checkins_clean.apply(lambda row: row.sum(), axis=1)

checkins_clean.sort_values(['total_checkins'], ascending=False, inplace=True)
checkins_clean.reset_index(inplace=True)
checkins_clean.head(3)
time_marker('done!')

[23:08:40.205283] group by business_id....
[23:08:44.233720] relabeling columns...
[23:08:44.234547] sorting columns...
[23:08:44.495465] collecting total count of checkins for business...
[23:09:07.465858] done!


# `bid_prefix` column

In [13]:
time_marker('appending bid_prefix column...')
checkins_clean['bid_prefix'] = checkins_clean.business_id.apply(lambda x: x[:1])
checkins_clean.head(8)

[23:09:07.475658] appending bid_prefix column...


,business_id,d0_h0,d0_h1,d0_h2,d0_h3,d0_h4,d0_h5,d0_h6,d0_h7,d0_h8,...,d6_h16,d6_h17,d6_h18,d6_h19,d6_h20,d6_h21,d6_h22,d6_h23,total_checkins,bid_prefix
0,FaHADZARwnY4yvlvpnsfGA,1326.0,67.0,273.0,683.0,969.0,1003.0,1044.0,1158.0,1203.0,...,810.0,604.0,461.0,353.0,309.0,226.0,139.0,29.0,126689.0,F
1,JmI9nslLD7KZqRr__Bg6NQ,1080.0,32.0,192.0,590.0,845.0,1117.0,1377.0,1264.0,921.0,...,511.0,391.0,296.0,293.0,154.0,74.0,23.0,10.0,107891.0,J
2,yQab5dxZzgBLTEHCw9V7_w,453.0,441.0,471.0,601.0,496.0,553.0,445.0,422.0,451.0,...,135.0,49.0,31.0,9.0,3.0,6.0,13.0,71.0,47908.0,y
3,5LNZ67Yw9RD6nf4_UhXOjw,418.0,84.0,62.0,30.0,40.0,81.0,157.0,167.0,213.0,...,570.0,666.0,703.0,723.0,649.0,608.0,465.0,312.0,43026.0,5
4,SMPbvZLSMMb7KU76YNYMGg,291.0,180.0,77.0,53.0,49.0,57.0,129.0,221.0,213.0,...,341.0,329.0,316.0,284.0,291.0,259.0,234.0,170.0,31864.0,S
5,Wxxvi3LZbHNIDwJ-ZimtnA,283.0,43.0,32.0,24.0,30.0,53.0,108.0,116.0,153.0,...,388.0,415.0,393.0,367.0,272.0,263.0,200.0,131.0,29744.0,W
6,IZivKqtHyz4-ts8KsnvMrA,366.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,...,380.0,425.0,450.0,451.0,346.0,140.0,10.0,2.0,29657.0,I
7,na4Th5DrNauOv-c43QQFvA,323.0,49.0,41.0,27.0,34.0,50.0,78.0,123.0,139.0,...,402.0,356.0,394.0,357.0,326.0,301.0,218.0,109.0,28458.0,n


# Write to Files

In [14]:
time_marker(text='Writing to files...')
file_count = len(checkins_clean.bid_prefix.unique())

for i, prefix in enumerate(sorted(checkins_clean.bid_prefix.unique())):
    df = checkins_clean[checkins_clean.bid_prefix == prefix].iloc[:,:-1].copy()
    df.reset_index(inplace=True, drop=True)
    file_name = '../clean_data/checkins/{}_{}_checkins_clean.csv'.format(str(i).zfill(2), prefix)
    time_marker(text='Writing {:d} records to file {}'.format(df.shape[0], file_name))
    if DRY_RUN:
        pass
    else:
        df.to_csv(file_name, encoding='utf-8')
time_marker(text='Done!')

[23:09:07.630973] writing to files...
[23:09:07.709048] writing 2138 records to file ../clean_data/checkins/00_-_checkins_clean.csv
[23:09:08.155428] writing 2108 records to file ../clean_data/checkins/01_0_checkins_clean.csv
[23:09:08.564122] writing 2067 records to file ../clean_data/checkins/02_1_checkins_clean.csv
[23:09:08.969069] writing 2037 records to file ../clean_data/checkins/03_2_checkins_clean.csv
[23:09:09.378367] writing 2131 records to file ../clean_data/checkins/04_3_checkins_clean.csv
[23:09:09.797262] writing 2117 records to file ../clean_data/checkins/05_4_checkins_clean.csv
[23:09:10.208894] writing 2131 records to file ../clean_data/checkins/06_5_checkins_clean.csv
[23:09:10.622841] writing 2117 records to file ../clean_data/checkins/07_6_checkins_clean.csv
[23:09:11.024672] writing 2132 records to file ../clean_data/checkins/08_7_checkins_clean.csv
[23:09:11.439184] writing 2048 records to file ../clean_data/checkins/09_8_checkins_clean.csv
[23:09:11.860997] writ